# Fase extra:

### Importações de bibliotecas e carregamento dos dados

In [2]:
import pandas as pd
import numpy as np

# carregando dados
print("Carregando os datasets...")
original_path = '../data/raw/desafio_indicium_imdb.csv'
budget_path = '../data/external_data/tmdb_5000_movies.csv'
cpi_path = '../data/external_data/cpi_data.csv'

# Carregando os datasets
df_original = pd.read_csv(original_path)
df_original = df_original.iloc[:, 1:] # remocao da coluna de índice

df_budget = pd.read_csv(budget_path)
df_cpi = pd.read_csv(cpi_path)
print("Datasets carregados com sucesso !!")

Carregando os datasets...
Datasets carregados com sucesso !!


### INTEGRAÇÃO DOS DADOS

In [3]:
print("\nIntegrando dataset de orçamento...")
# Juntando o dataset original com o de orçamento pelo título do filme
df_merged = pd.merge(df_original, df_budget[['title', 'budget']], left_on='Series_Title', right_on='title', how='left')

# Limpando o novo dataframe
df_merged = df_merged.drop(columns=['title'])
#  valor 0 --> NaN.
df_merged['budget'] = df_merged['budget'].replace(0, np.nan)

print(f"{df_merged['budget'].notna().sum()} de {len(df_merged)} filmes tiveram um orçamento correspondente.")


Integrando dataset de orçamento...
424 de 999 filmes tiveram um orçamento correspondente.


### AJUSTE PELA INFLAÇÃO E CÁLCULO DO ROI

In [4]:
print("\nIniciando cálculo de ROI e ajuste pela inflação...")

# Limpeza das colunas monetárias e de ano
df_merged['Gross'] = pd.to_numeric(df_merged['Gross'].astype(str).str.replace(',', ''), errors='coerce')
df_merged['Released_Year'] = pd.to_numeric(df_merged['Released_Year'], errors='coerce')
df_merged.dropna(subset=['Released_Year'], inplace=True)
df_merged['Released_Year'] = df_merged['Released_Year'].astype(int)

# Função para ajustar um valor monetário para dólares de 2024
def adjust_for_inflation(row, column_name, cpi_df):
    year = row['Released_Year']
    value = row[column_name]

    if pd.isna(value) or not (cpi_df['Year'].min() <= year <= cpi_df['Year'].max()):
        return np.nan

    cpi_movie_year = cpi_df.loc[cpi_df['Year'] == year, 'CPI'].iloc[0]
    cpi_current_year = cpi_df.loc[cpi_df['Year'] == 2024, 'CPI'].iloc[0]

    inflation_ratio = cpi_current_year / cpi_movie_year
    return value * inflation_ratio



Iniciando cálculo de ROI e ajuste pela inflação...


In [5]:
# Aplicando a função para criar as colunas ajustadas
df_merged['Gross_Adjusted'] = df_merged.apply(lambda row: adjust_for_inflation(row, 'Gross', df_cpi), axis=1)
df_merged['Budget_Adjusted'] = df_merged.apply(lambda row: adjust_for_inflation(row, 'budget', df_cpi), axis=1)

# ROI = (Lucro / Custo) = (Faturamento - Orçamento) / Orçamento
df_merged['ROI'] = (df_merged['Gross_Adjusted'] - df_merged['Budget_Adjusted']) / df_merged['Budget_Adjusted']
print("Cálculos finalizados.")

Cálculos finalizados.


### VISUALIZAÇÃO DOS RESULTADOS

In [6]:
print("\n" + "="*50)
print("--- Análise Final com Dados Externos ---")
# renomeando colunas para a visualização
df_final = df_merged[[
    'Series_Title',
    'Released_Year',
    'Gross',
    'Gross_Adjusted',
    'budget',
    'Budget_Adjusted',
    'ROI'
]].copy()
df_final.rename(columns={'budget': 'Budget_Original'}, inplace=True)

# Exibindo os 20 filmes com maior ROI (que possuem dados de orçamento)
print("\nTop 10 Filmes por Retorno sobre o Investimento (ROI):")
display(df_final.dropna(subset=['ROI']).sort_values(by='ROI', ascending=False).head(20))
print("="*50)



--- Análise Final com Dados Externos ---

Top 10 Filmes por Retorno sobre o Investimento (ROI):


,Series_Title,Released_Year,Gross,Gross_Adjusted,Budget_Original,Budget_Adjusted,ROI
50,Modern Times,1936,163245.0,3.684040e+06,1.0,2.256755e+01,163244.000000
843,Halloween,1978,47000000.0,2.261255e+08,300000.0,1.443354e+06,155.666667
278,Rocky,1976,117235247.0,6.463165e+08,1000000.0,5.512988e+06,116.235247
815,Clerks,1994,3151130.0,6.669871e+06,27000.0,5.714982e+04,115.708519
616,Once,2007,9439923.0,1.428461e+07,160000.0,2.421140e+05,57.999519
313,Gone with the Wind,1939,198676459.0,4.483642e+09,4000000.0,9.027022e+07,48.669115
931,Saw,2004,56000369.0,9.299470e+07,1200000.0,1.992731e+06,45.666974
847,Blazing Saddles,1974,119500000.0,7.603618e+08,2600000.0,1.654344e+07,44.961538
687,E.T. the Extra-Terrestrial,1982,435110554.0,1.414398e+09,10500000.0,3.413196e+07,40.439100
48,Psycho,1960,32000000.0,3.391232e+08,806948.0,8.551713e+06,38.655591


### Salvar o Dataset Enriquecido

In [7]:
# Definindo o caminho para a pasta de dados processados
output_path = '../data/processed/imdb_com_roi.csv'

# Salvando o DataFrame final como um novo arquivo CSV
df_final.to_csv(output_path, index=False)

print(f"Dataset final com dados de orçamento e ROI salvo com sucesso em: {output_path}")

Dataset final com dados de orçamento e ROI salvo com sucesso em: ../data/processed/imdb_com_roi.csv
